Version 1.1.0

# Mean encodings

In this programming assignment you will be working with `1C` dataset from the final competition. You are asked to encode `item_id` in 4 different ways:

    1) Via KFold scheme;  
    2) Via Leave-one-out scheme;
    3) Via smoothing scheme;
    4) Via expanding mean scheme.

**You will need to submit** the correlation coefficient between resulting encoding and target variable up to 4 decimal places.

### General tips

* Fill NANs in the encoding with `0.3343`.
* Some encoding schemes depend on sorting order, so in order to avoid confusion, please use the following code snippet to construct the data frame. This snippet also implements mean encoding without regularization.

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from grader import Grader

# Read data

In [2]:
sales = pd.read_csv('../readonly/final_project_data/sales_train.csv.gz')

In [3]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
sales.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [5]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
date              object
date_block_num    int64
shop_id           int64
item_id           int64
item_price        float64
item_cnt_day      float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [6]:
sales.shape

(2935849, 6)

# Aggregate data

Since the competition task is to make a monthly prediction, we need to aggregate the data to montly level before doing any encodings. The following code-cell serves just that purpose.

In [7]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [8]:
all_data.head()

,shop_id,item_id,date_block_num,target
139255,0,19,0,0.0
141495,0,27,0,0.0
144968,0,28,0,0.0
142661,0,29,0,0.0
138947,0,32,0,6.0


In [9]:
all_data.shape

(10913850, 4)

In [10]:
all_data.nunique()

shop_id              60
item_id           21807
date_block_num       34
target              446
dtype: int64

# Mean encodings without regularization

After we did the techinical work, we are ready to actually *mean encode* the desired `item_id` variable. 

Here are two ways to implement mean encoding features *without* any regularization. You can use this code as a starting point to implement regularized techniques. 

#### Method 1

In [11]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.483038698862


In [12]:
item_id_target_mean.head()

item_id
0    0.020000
1    0.023810
2    0.019802
3    0.019802
4    0.020000
Name: target, dtype: float64

In [13]:
all_data["item_id"].head()

139255    19
141495    27
144968    28
142661    29
138947    32
Name: item_id, dtype: int32

#### Method 2

In [14]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.483038698862


In [15]:
all_data.groupby('item_id')['target'].sum()


item_id
0           1.0
1           6.0
2           2.0
3           2.0
4           1.0
5           1.0
6           1.0
7           1.0
8           2.0
9           1.0
10          1.0
11          1.0
12          1.0
13          1.0
14          1.0
15          1.0
16          1.0
17          1.0
18          1.0
19          1.0
20          1.0
21          1.0
22          1.0
23          1.0
24          1.0
25          1.0
26          1.0
27         42.0
28         84.0
29         12.0
          ...  
22140     287.0
22141      53.0
22142      10.0
22143     865.0
22144     116.0
22145     320.0
22146      29.0
22147      61.0
22148       2.0
22149      35.0
22150      58.0
22151     371.0
22152      85.0
22153       9.0
22154      59.0
22155      74.0
22156       4.0
22157       4.0
22158       1.0
22159      15.0
22160      49.0
22161       1.0
22162     699.0
22163      75.0
22164     493.0
22165       2.0
22166     348.0
22167    1320.0
22168       6.0
22169       1.0
Name: target, Le

In [16]:
all_data.groupby('item_id')['target'].transform('mean')

139255      0.022222
141495      0.056834
144968      0.141176
142661      0.037383
138947      1.319042
138948      0.527112
138949      0.146108
139247      0.944681
142672      0.070943
142065      0.085828
139208      0.070596
142670      0.032847
139207      0.086773
138950      0.110971
143764      0.058450
141505      0.076040
139199      0.069005
138952      0.116646
139176      0.044444
138951      0.148802
139177      0.067236
139178      0.119798
139179      0.073126
143769      0.112575
142671      0.052989
144539      0.098361
139180      0.157485
138953      0.071168
144265      0.043796
141744      0.056250
              ...   
10772600    0.830357
10770510    0.140000
10769953    0.502326
10769955    1.362817
10768833    0.163556
10769961    0.370044
10770625    0.159066
10769956    0.699195
10771598    1.937198
10767854    2.173392
10768086    3.324716
10768087    0.751576
10768088    1.317150
10767847    2.267442
10769954    1.003861
10767848    6.594595
10767849    0

See the printed value? It is the correlation coefficient between the target variable and your new encoded feature. You need to **compute correlation coefficient** between the encodings, that you will implement and **submit those to coursera**.

In [17]:
grader = Grader()

# 1. KFold scheme

Explained starting at 41 sec of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

**Now it's your turn to write the code!** 

You may use 'Regularization' video as a reference for all further tasks.

First, implement KFold scheme with five folds. Use KFold(5) from sklearn.model_selection. 

1. Split your data in 5 folds with `sklearn.model_selection.KFold` with `shuffle=False` argument.
2. Iterate through folds: use all but the current fold to calculate mean target for each level `item_id`, and  fill the current fold.

    *  See the **Method 1** from the example implementation. In particular learn what `map` and pd.Series.map functions do. They are pretty handy in many situations.

In [18]:
from sklearn.model_selection import KFold

kf = KFold(5, shuffle=False)
kf.get_n_splits(all_data)

5

In [19]:
for train_idx, test_idx in kf.split(all_data):
    #X_train, X_test = all_data[train_idx], all_data[test_idx]
    print(train_idx.shape)
    print(test_idx.shape)
    #print(all_data.iloc[train_idx])
    break

(8731080,)
(2182770,)


In [20]:
# loc : 인덱스가 0과 1걸 읽어옴
all_data.loc[[0,1]]

,shop_id,item_id,date_block_num,target,item_target_enc
0,59,22154,0,1.0,0.109870
1,59,2552,0,0.0,0.018182


In [21]:
# iloc : 첫번째 인덱스부터 순서대로 불러옴
all_data.iloc[[0,1]]

,shop_id,item_id,date_block_num,target,item_target_enc
139255,0,19,0,0.0,0.022222
141495,0,27,0,0.0,0.056834


In [22]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_target_enc
139255,0,19,0,0.0,0.022222
141495,0,27,0,0.0,0.056834
144968,0,28,0,0.0,0.141176
142661,0,29,0,0.0,0.037383
138947,0,32,0,6.0,1.319042


In [23]:
# YOUR CODE GOES HERE

for train_idx, val_idx in kf.split(all_data):
    #print(train_idx)
    #print(test_idx)
    #print("-------------------------------------------")
    X_train, X_val = all_data.loc[train_idx], all_data.loc[val_idx]
    #print(X_train.head())
    #print(X_val.head())   
    #print("-------------------------------------------")
    #break
    #print(X_val.index)
    #print(all_data.loc[X_val.index].head())
    #break
    target_mean = X_train.groupby("item_id")["target"].mean()    
    all_data.loc[X_val.index]["item_target_enc"] = X_val["item_id"].map(target_mean)

all_data['item_target_enc'].fillna(0.3343, inplace=True) 
encoded_feature = all_data['item_target_enc'].values
# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('KFold_scheme', corr)

0.483038698862
Current answer for task KFold_scheme is: 0.483038698862


# 2. Leave-one-out scheme

Now, implement leave-one-out scheme. Note that if you just simply set the number of folds to the number of samples and run the code from the **KFold scheme**, you will probably wait for a very long time. 

To implement a faster version, note, that to calculate mean target value using all the objects but one *given object*, you can:

1. Calculate sum of the target values using all the objects.
2. Then subtract the target of the *given object* and divide the resulting value by `n_objects - 1`. 

Note that you do not need to perform `1.` for every object. And `2.` can be implemented without any `for` loop.

It is the most convenient to use `.transform` function as in **Method 2**.

In [24]:
print(all_data.groupby("item_id")["target"].sum().head())    
print(all_data.groupby("item_id")["target"].transform('sum').head())

item_id
0    1.0
1    6.0
2    2.0
3    2.0
4    1.0
Name: target, dtype: float64
139255       1.0
141495      42.0
144968      84.0
142661      12.0
138947    2092.0
Name: target, dtype: float64


In [25]:
# YOUR CODE GOES HERE
target_sum = all_data.groupby("item_id")["target"].transform("sum")
target_count = all_data.groupby("item_id")["target"].transform("count")

all_data["item_target_enc"]  = (target_sum - all_data["target"])/(target_count - 1)

all_data['item_target_enc'].fillna(0.3343, inplace=True) 
encoded_feature = all_data['item_target_enc'].values
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Leave-one-out_scheme', corr)

0.480384831129
Current answer for task Leave-one-out_scheme is: 0.480384831129


# 3. Smoothing

Explained starting at 4:03 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Next, implement smoothing scheme with $\alpha = 100$. Use the formula from the first slide in the video and $0.3343$ as `globalmean`. Note that `nrows` is the number of objects that belong to a certain category (not the number of rows in the dataset).

In [26]:
# YOUR CODE GOES HERE
n_rows = all_data.groupby("item_id")["target"].transform("count")
target_mean = all_data.groupby("item_id")["target"].transform("mean")
global_mean = 0.3343
alpha = 100

all_data["item_target_enc"]  = (target_mean * n_rows + global_mean * alpha)/(n_rows + alpha)
all_data['item_target_enc'].fillna(0.3343, inplace=True) 
encoded_feature = all_data['item_target_enc'].values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Smoothing_scheme', corr)

0.48181987971
Current answer for task Smoothing_scheme is: 0.48181987971


# 4. Expanding mean scheme

Explained starting at 5:50 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Finally, implement the *expanding mean* scheme. It is basically already implemented for you in the video, but you can challenge yourself and try to implement it yourself. You will need [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) and [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html) functions from pandas.

In [27]:
# YOUR CODE GOES HERE
cumsum = all_data.groupby("item_id")["target"].cumsum() - all_data["target"]
cumcnt = all_data.groupby("item_id").cumcount()

all_data["item_target_enc"]  = cumsum/cumcnt
all_data['item_target_enc'].fillna(0.3343, inplace=True) 
encoded_feature = all_data['item_target_enc'].values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Expanding_mean_scheme', corr)

0.502524521108
Current answer for task Expanding_mean_scheme is: 0.502524521108


## Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. Note: Token expires 30 minutes after generation.

In [28]:
STUDENT_EMAIL = ""# EMAIL HERE
STUDENT_TOKEN = ""# TOKEN HERE
grader.status()

You want to submit these numbers:
Task KFold_scheme: 0.483038698862
Task Leave-one-out_scheme: 0.480384831129
Task Smoothing_scheme: 0.48181987971
Task Expanding_mean_scheme: 0.502524521108


In [29]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

TypeError: argument of type 'NoneType' is not iterable